In [ ]:
import time
from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()
spark.conf.set(
    "spark.sql.shuffle.partitions", "400"
)  # Default is 200, increase if needed


path_n='gs://open-targets-data-releases/25.03/output/'

target = spark.read.parquet(f"{path_n}target/")

diseases = spark.read.parquet(f"{path_n}disease/")

evidences = spark.read.parquet(f"{path_n}evidence")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

datasource_filter = [
    "ot_genetics_portal",
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)

print("run temporary direction of effect")


print("built drugApproved dataset")

analysis_chembl_indication = (
    discrepancifier(
        assessment.filter((F.col("datasourceId") == "chembl"))
        .withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
    # .persist()
)

chemblAssoc = (
    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .count()
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

print("built chemblAssoc dataset")

benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            analysis_chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    ).filter(
        F.col("name") != "COVID-19"
    )  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")


print("built benchmark dataset")

#### Analysis

#### 1 Build a dictionary with the distinct values as key and column names as value
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

# Initialize an empty dictionary
disdic = {}

# Iterate over the list of column names
for col_name in variables_study:
    # Extract distinct values for the column
    distinct_values = benchmark.select(col_name).distinct().collect()

    # Populate the dictionary
    for row in distinct_values:
        distinct_value = row[col_name]
        if distinct_value is not None:  # Exclude None (null) values
            disdic[distinct_value] = col_name

####2 Define agregation function
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *


def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st


#####3 run in a function
def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):
    wComparison = Window.partitionBy(comparisonColumn)
    wPrediction = Window.partitionBy(predictionColumn)
    wPredictionComparison = Window.partitionBy(comparisonColumn, predictionColumn)
    results = []
    # uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("dataset", F.lit(data))
        .withColumn("predictionType", F.lit(predictionType))
        # .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn("comparisonColumn", F.lit(comparisonColumn))
        .withColumn("predictionColumnValue", F.lit(predictionColumn))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "dataset",
            "comparisonColumn",
            "predictionColumnValue",
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )

    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )

    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    path = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + comparisonType
        + "_"
        + predictionColumn
        + ".parquet"
    )
    print(path)
    
    ### making analysis
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    results.extend(
        [
            comparisonType,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            # studies,
            # tissues,
            path,
        ]
    )
    return results


#### 3 Loop over different datasets (as they will have different rows and columns)


def comparisons_df_iterative(elements):
    # toAnalysis = [(key, value) for key, value in disdic.items() if value == projectId]
    toAnalysis = [(col, "predictor") for col in elements]
    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(toAnalysis, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ('Phase>=3','clinical'),
            ('Phase>=2','clinical'),
            ('Phase>=1','clinical'),
            ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()


print("load comparisons_df_iterative function")


full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
print("created full_data and lists")

rightTissue = spark.read.csv(
    'gs://ot-team/jroldan/analysis/20250402_rightTissueListMarchRelease.csv',
    header=True,
).drop("_c0")

print("loaded rightTissue dataset")

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

print("built negativeTD dataset")

bench2 = benchmark.join(
    rightTissue, on=["name", "bioSampleName"], how="left"
).withColumn(
    "rightTissue",
    F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
)

print("built bench2 dataset")

###### cut from here
print("looping for variables_study")
# List of columns to analyze
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

# Dictionary to store results
pivoted_dfs = {}

# Loop over the columns
for col in variables_study:
    window_spec = Window.partitionBy("targetId", "diseaseId", col).orderBy(
        F.col("pValueExponent").asc()
    )
    print(f"Processing: {col}")

    pivoted_df = (
        bench2.withColumn(
            "rightTissue",
            F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        )
        .withColumn(
            "isRightTissueSignalAgreed",
            F.collect_set(
                F.when(F.col("rightTissue") == "yes", F.col("agree_lowestPval"))
            ).over(window_spec),
        )
        .withColumn(
            "isSignalFromRightTissue",
            F.first(
                F.when(
                    F.col("AgreeDrug") == F.col("agree_lowestPval"),
                    F.col("rightTissue"),
                ),
                ignorenulls=True,
            ).over(window_spec),
        )
        .groupBy(
            "targetId",
            "diseaseId",
            "maxClinPhase",
            "rightTissue",
            "isRightTissueSignalAgreed",
            "isSignalFromRightTissue",
        )
        .pivot(col)  # Pivot the column dynamically
        .agg(F.collect_set("agree_lowestPval"))
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(
                F.lit("no")
            ),
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        )
        .select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.array_contains(F.col(x), "yes"), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}_only")
                    for x, value in [
                        (key, val) for key, val in disdic.items() if val == col
                    ]
                ]
            )
            + (
                [
                    F.when(
                        F.array_contains(F.col("isRightTissueSignalAgreed"), "yes"),
                        F.lit("yes"),
                    )
                    .otherwise(F.lit("no"))
                    .alias(f"{x}_isRightTissueSignalAgreed")
                    for x, value in [
                        (key, val) for key, val in disdic.items() if val == col
                    ]
                ]
            )
        )
    )  # Collect unique values

    # Store the DataFrame in the dictionary
    pivoted_dfs[col] = pivoted_df





''' 
result = []
result_st = []
result_ci = []
array2 = []
listado = []
result_all = []
today_date = str(date.today())
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

##### PROJECT ID ###### 
print('working with projectId')
pivoted_dfs['projectId'].persist()
unique_values = benchmark.select('projectId').distinct().rdd.flatMap(lambda x: x).collect()
filter = len(pivoted_dfs['projectId'].drop(*unique_values).columns[12:])
print('There are ', filter, 'columns to analyse with phases')
rows = comparisons_df_iterative(pivoted_dfs['projectId'].columns[-filter:])

# If needed, now process the rest
for row in rows:
    results = aggregations_original(
        pivoted_dfs['projectId'], "propagated", listado, *row, today_date
    )
    result_all.append(results)

pivoted_dfs['projectId'].unpersist()
print('df unpersisted')

##### BIOSAMPLE NAME ###### 
print('working with biosampleName')
pivoted_dfs['biosampleName'].persist()
unique_values = benchmark.select('biosampleName').distinct().rdd.flatMap(lambda x: x).collect()
filter = len(pivoted_dfs['biosampleName'].drop(*unique_values).columns[12:])
print('There are ', filter, 'columns to analyse with phases')
rows = comparisons_df_iterative(pivoted_dfs['biosampleName'].columns[-filter:])

for row in rows:
    results = aggregations_original(
        pivoted_dfs['biosampleName'], "propagated", listado, *row, today_date
    )
    result_all.append(results)

pivoted_dfs['biosampleName'].unpersist()
print('df unpersisted')

##### RIGHTSTUDYTYPE  ###### 
print('working with rightStudyType')
pivoted_dfs['rightStudyType'].persist()
unique_values = benchmark.select('rightStudyType').distinct().rdd.flatMap(lambda x: x).collect()
filter = len(pivoted_dfs['rightStudyType'].drop(*unique_values).columns[12:])
print('There are ', filter, 'columns to analyse with phases')
rows = comparisons_df_iterative(pivoted_dfs['rightStudyType'].columns[-filter:])

for row in rows:
    results = aggregations_original(
        pivoted_dfs['rightStudyType'], "propagated", listado, *row, today_date
    )
    result_all.append(results)
pivoted_dfs['rightStudyType'].unpersist()
print('df unpersisted')

##### COLOC DOE ######
print('working with colocDoE')
pivoted_dfs['colocDoE'].persist()
unique_values = benchmark.select('colocDoE').distinct().rdd.flatMap(lambda x: x).collect()
filter = len(pivoted_dfs['colocDoE'].drop(*unique_values).columns[12:])
print('There are ', filter, 'columns to analyse with phases')
rows = comparisons_df_iterative(pivoted_dfs['colocDoE'].columns[-filter:])

for row in rows:
    results = aggregations_original(
        pivoted_dfs['colocDoE'], "propagated", listado, *row, today_date
    )
    result_all.append(results)
pivoted_dfs['colocDoE'].unpersist()
print('df unpersisted')

schema = StructType(
    [
        StructField("group", StringType(), True),
        StructField("comparison", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)
import re

# Define the list of patterns to search for
patterns = [
    "_only",
    #"_tissue",
    #"_isSignalFromRightTissue",
    "_isRightTissueSignalAgreed",
]
# Create a regex pattern to match any of the substrings
regex_pattern = "(" + "|".join(map(re.escape, patterns)) + ")"

# Convert list of lists to DataFrame
df = (
    spreadSheetFormatter(spark.createDataFrame(result_all, schema=schema))
    .withColumn(
        "prefix",
        F.regexp_replace(
            F.col("comparison"), regex_pattern + ".*", ""
        ),  # Extract part before the pattern
    )
    .withColumn(
        "suffix",
        F.regexp_extract(
            F.col("comparison"), regex_pattern, 0
        ),  # Extract the pattern itself
    )
)

### annotate projectId, tissue, qtl type and doe type:

from pyspark.sql.functions import create_map
from itertools import chain

mapping_expr=create_map([F.lit(x) for x in chain(*disdic.items())])

df_annot=df.withColumn('annotation',mapping_expr.getItem(F.col('prefix')))

df_annot.toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_credibleSetColocDoEanalysis_RightTissues.csv"
)

print("dataframe written \n Analysis finished")
'''

spark session created at 2025-05-16 20:31:59.245545
Analysis started on 2025-05-16 at  2025-05-16 20:31:59.245545


25/05/16 20:32:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


loaded files
loaded newColoc


loaded gwasComplete
loaded resolvedColloc
run temporary direction of effect
built drugApproved dataset


built chemblAssoc dataset
built benchmark dataset


load comparisons_df_iterative function
created full_data and lists
loaded rightTissue dataset
built negativeTD dataset
built bench2 dataset
looping for variables_study
Processing: projectId


Processing: biosampleName


Processing: rightStudyType


Processing: colocDoE


' \nresult = []\nresult_st = []\nresult_ci = []\narray2 = []\nlistado = []\nresult_all = []\ntoday_date = str(date.today())\nvariables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]\n\n##### PROJECT ID ###### \nprint(\'working with projectId\')\npivoted_dfs[\'projectId\'].persist()\nunique_values = benchmark.select(\'projectId\').distinct().rdd.flatMap(lambda x: x).collect()\nfilter = len(pivoted_dfs[\'projectId\'].drop(*unique_values).columns[12:])\nprint(\'There are \', filter, \'columns to analyse with phases\')\nrows = comparisons_df_iterative(pivoted_dfs[\'projectId\'].columns[-filter:])\n\n# If needed, now process the rest\nfor row in rows:\n    results = aggregations_original(\n        pivoted_dfs[\'projectId\'], "propagated", listado, *row, today_date\n    )\n    result_all.append(results)\n\npivoted_dfs[\'projectId\'].unpersist()\nprint(\'df unpersisted\')\n\n##### BIOSAMPLE NAME ###### \nprint(\'working with biosampleName\')\npivoted_dfs[\'biosampleName\

In [13]:
### dataframe Study X biosampleName

window_spec = Window.partitionBy("targetId", "diseaseId", 'projectId','biosampleName').orderBy(
    F.col("pValueExponent").asc()
)

pivoted_df = (
    bench2.withColumn(
        "rightTissue",
        F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
    )
    .withColumn(
        "agree_lowestPval",
        F.first("AgreeDrug", ignorenulls=True).over(
            window_spec
        ),  ### ignore nulls aded 29.01.2025
        #### take directionality from lowest p value
    )
    .withColumn(
        "isRightTissueSignalAgreed",
        F.collect_set(
            F.when(F.col("rightTissue") == "yes", F.col("agree_lowestPval"))
        ).over(window_spec),
    )
    .withColumn(
        "isSignalFromRightTissue",
        F.first(
            F.when(
                F.col("AgreeDrug") == F.col("agree_lowestPval"),
                F.col("rightTissue"),
            ),
            ignorenulls=True,
        ).over(window_spec),
    )
    .groupBy(
        "targetId",
        "diseaseId",
        "maxClinPhase",
        #"rightTissue",
        #"isRightTissueSignalAgreed",
        #"isSignalFromRightTissue",
        'biosampleName'
    )
    .pivot('projectId')  # Pivot the column dynamically
    .agg(F.collect_set("agree_lowestPval"))
)    

# Store the DataFrame in the dictionary
pivoted_dfs['projectBiosample'] = pivoted_df

In [18]:
# 1. Create a struct of the columns you want to combine
df_combined = pivoted_dfs['projectBiosample'].withColumn("combined", F.struct("targetId", "diseaseId"))

# 2. Use dense_rank over a window partitioned by the combined struct
window_spec = Window.orderBy("combined")
df_with_rank = df_combined.withColumn("unique_id", F.dense_rank().over(window_spec))

# 3. Drop the intermediate 'combined' column (optional)
df_final = df_with_rank.drop("combined")

#pivoted_dfs['projectBiosample'].withColumn("unique_id", F.dense_rank().over(window_spec)).show()

In [24]:
array_columns = [field.name for field in df_final.schema if isinstance(field.dataType, ArrayType)]

In [25]:
array_columns

['Alasoo_2018',
 'Aygun_2021',
 'BLUEPRINT',
 'Bossini-Castillo_2019',
 'BrainSeq',
 'Braineac2',
 'CAP',
 'CEDAR',
 'CommonMind',
 'Cytoimmgen',
 'FUSION',
 'Fairfax_2014',
 'GENCORD',
 'GEUVADIS',
 'GTEx',
 'Gilchrist_2021',
 'HipSci',
 'Jerber_2021',
 'Kim-Hellmuth_2017',
 'Lepik_2017',
 'Nathan_2022',
 'Nedelec_2016',
 'OneK1K',
 'PISA',
 'Peng_2018',
 'Perez_2022',
 'PhLiPS',
 'Quach_2016',
 'ROSMAP',
 'Randolph_2021',
 'Schmiedel_2018',
 'Schwartzentruber_2018',
 'Sun_2018',
 'TwinsUK',
 'UKB_PPP_EUR',
 'Walker_2019',
 'Young_2019',
 'iPSCORE',
 'van_de_Bunt_2015']

In [27]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

def transform_array_column_native_fixed(df, col_name):
    """
    Transforms the content of an array column using native Spark functions with consistent output type.
    """
    return df.withColumn(
        col_name + "_transformed",
        F.when(F.size(F.col(col_name)) == 0, F.lit("0"))
        .when(F.array_contains(F.col(col_name), "no") & F.array_contains(F.col(col_name), "yes"), F.lit("0"))
        .when(F.array_contains(F.col(col_name), "no"), F.lit("1"))
        .when(F.array_contains(F.col(col_name), "yes"), F.lit("2"))
        .when(F.size(F.col(col_name)) == 1, F.element_at(F.col(col_name), 1))
        .otherwise(F.array_join(F.col(col_name), ",")) # Or handle other arrays differently as strings
    )

# Identify array columns
array_columns = [field.name for field in df_final.schema if isinstance(field.dataType, ArrayType)]

# Transform each array column
for col_name in array_columns:
    df_final = transform_array_column_native_fixed(df_final, col_name)
    df_final = df_final.drop(col_name).withColumnRenamed(col_name + "_transformed", col_name)

df_final.printSchema()
df_final.show(truncate=False)

root
 |-- targetId: string (nullable = true)
 |-- diseaseId: string (nullable = true)
 |-- maxClinPhase: double (nullable = true)
 |-- biosampleName: string (nullable = true)
 |-- unique_id: integer (nullable = false)
 |-- Alasoo_2018: string (nullable = true)
 |-- Aygun_2021: string (nullable = true)
 |-- BLUEPRINT: string (nullable = true)
 |-- Bossini-Castillo_2019: string (nullable = true)
 |-- BrainSeq: string (nullable = true)
 |-- Braineac2: string (nullable = true)
 |-- CAP: string (nullable = true)
 |-- CEDAR: string (nullable = true)
 |-- CommonMind: string (nullable = true)
 |-- Cytoimmgen: string (nullable = true)
 |-- FUSION: string (nullable = true)
 |-- Fairfax_2014: string (nullable = true)
 |-- GENCORD: string (nullable = true)
 |-- GEUVADIS: string (nullable = true)
 |-- GTEx: string (nullable = true)
 |-- Gilchrist_2021: string (nullable = true)
 |-- HipSci: string (nullable = true)
 |-- Jerber_2021: string (nullable = true)
 |-- Kim-Hellmuth_2017: string (nullable =

25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 21:16:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/16 2

+---------------+-------------+------------+------------------------------+---------+-----------+----------+---------+---------------------+--------+---------+---+-----+----------+----------+------+------------+-------+--------+----+--------------+------+-----------+-----------------+----------+-----------+------------+------+----+---------+----------+------+----------+------+-------------+--------------+---------------------+--------+-------+-----------+-----------+----------+-------+----------------+
|targetId       |diseaseId    |maxClinPhase|biosampleName                 |unique_id|Alasoo_2018|Aygun_2021|BLUEPRINT|Bossini-Castillo_2019|BrainSeq|Braineac2|CAP|CEDAR|CommonMind|Cytoimmgen|FUSION|Fairfax_2014|GENCORD|GEUVADIS|GTEx|Gilchrist_2021|HipSci|Jerber_2021|Kim-Hellmuth_2017|Lepik_2017|Nathan_2022|Nedelec_2016|OneK1K|PISA|Peng_2018|Perez_2022|PhLiPS|Quach_2016|ROSMAP|Randolph_2021|Schmiedel_2018|Schwartzentruber_2018|Sun_2018|TwinsUK|UKB_PPP_EUR|Walker_2019|Young_2019|iPSCORE|va

In [10]:
### mmatrix Study X biosampleName
pivoted_dfs['projectBiosample'].withColumn(
    "unique_batch_id",F.monotonically_increasing_id().over(Window.partitionBy('targetId','diseaseId'))).show()

AnalysisException: [UNSUPPORTED_EXPR_FOR_WINDOW] Expression "monotonically_increasing_id()" not supported within a window function.;
Project [targetId#1758, diseaseId#1911, maxClinPhase#6142, rightTissue#22416, isRightTissueSignalAgreed#22515, isSignalFromRightTissue#22567, biosampleName#1060, Alasoo_2018#22887, Aygun_2021#22889, BLUEPRINT#22891, Bossini-Castillo_2019#22893, BrainSeq#22895, Braineac2#22897, CAP#22899, CEDAR#22901, CommonMind#22903, Cytoimmgen#22905, FUSION#22907, Fairfax_2014#22909, GENCORD#22911, GEUVADIS#22913, GTEx#22915, Gilchrist_2021#22917, HipSci#22919, ... 23 more fields]
+- Project [targetId#1758, diseaseId#1911, maxClinPhase#6142, rightTissue#22416, isRightTissueSignalAgreed#22515, isSignalFromRightTissue#22567, biosampleName#1060, Alasoo_2018#22887, Aygun_2021#22889, BLUEPRINT#22891, Bossini-Castillo_2019#22893, BrainSeq#22895, Braineac2#22897, CAP#22899, CEDAR#22901, CommonMind#22903, Cytoimmgen#22905, FUSION#22907, Fairfax_2014#22909, GENCORD#22911, GEUVAD

In [2]:
import time
from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
# from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
# from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()
spark.conf.set(
    "spark.sql.shuffle.partitions", "400"
)  # Default is 200, increase if needed


path_n='gs://open-targets-data-releases/25.03/output/'

target = spark.read.parquet(f"{path_n}target/")

diseases = spark.read.parquet(f"{path_n}disease/")

evidences = spark.read.parquet(f"{path_n}evidence")

credible = spark.read.parquet(f"{path_n}credible_set")

new = spark.read.parquet(f"{path_n}colocalisation_coloc") 

index=spark.read.parquet(f"{path_n}study/")

variantIndex = spark.read.parquet(f"{path_n}variant")

biosample = spark.read.parquet(f"{path_n}biosample")

print("loaded files")

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
            'isTransQtl'
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        index.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId",
            "geneId",
            "projectId",
            "studyType as indexStudyType",
            "condition",
            "biosampleId",
        ),
        on="rightStudyId",
        how="left",
)
    # .persist()
)

print("loaded newColoc")

# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns)  # .persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
)  # .persist()

print("loaded gwasComplete")

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    ).withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    # .persist()
)
print("loaded resolvedColloc")

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

datasource_filter = [
    "ot_genetics_portal",
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path_n, datasource_filter
)

print("run temporary direction of effect")


print("built drugApproved dataset")

analysis_chembl_indication = (
    discrepancifier(
        assessment.filter((F.col("datasourceId") == "chembl"))
        .withColumn(
            "maxClinPhase",
            F.max(F.col("clinicalPhase")).over(
                Window.partitionBy("targetId", "diseaseId")
            ),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .agg(F.count("targetId"))
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
    # .persist()
)

chemblAssoc = (
    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId", "maxClinPhase")
        .pivot("homogenized")
        .count()
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

print("built chemblAssoc dataset")

benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            analysis_chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    ).filter(
        F.col("name") != "COVID-19"
    )  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")


print("built benchmark dataset")

#### Analysis

#### 1 Build a dictionary with the distinct values as key and column names as value
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

# Initialize an empty dictionary
disdic = {}

# Iterate over the list of column names
for col_name in variables_study:
    # Extract distinct values for the column
    distinct_values = benchmark.select(col_name).distinct().collect()

    # Populate the dictionary
    for row in distinct_values:
        distinct_value = row[col_name]
        if distinct_value is not None:  # Exclude None (null) values
            disdic[distinct_value] = col_name

####2 Define agregation function
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *


def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st


#####3 run in a function
def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):
    wComparison = Window.partitionBy(comparisonColumn)
    wPrediction = Window.partitionBy(predictionColumn)
    wPredictionComparison = Window.partitionBy(comparisonColumn, predictionColumn)
    results = []
    # uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("dataset", F.lit(data))
        .withColumn("predictionType", F.lit(predictionType))
        # .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn("comparisonColumn", F.lit(comparisonColumn))
        .withColumn("predictionColumnValue", F.lit(predictionColumn))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "dataset",
            "comparisonColumn",
            "predictionColumnValue",
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )

    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )

    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    path = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + comparisonType
        + "_"
        + predictionColumn
        + ".parquet"
    )
    print(path)
    
    ### making analysis
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    results.extend(
        [
            comparisonType,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            # studies,
            # tissues,
            path,
        ]
    )
    return results


#### 3 Loop over different datasets (as they will have different rows and columns)


def comparisons_df_iterative(elements):
    # toAnalysis = [(key, value) for key, value in disdic.items() if value == projectId]
    toAnalysis = [(col, "predictor") for col in elements]
    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(toAnalysis, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ('Phase>=3','clinical'),
            ('Phase>=2','clinical'),
            ('Phase>=1','clinical'),
            ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()


print("load comparisons_df_iterative function")


full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
print("created full_data and lists")

rightTissue = spark.read.csv(
    'gs://ot-team/jroldan/analysis/jroldan_analysis_20250526_curatedRightTissue.csv',
    header=True,
).drop("_c0")

print("loaded rightTissue dataset")

negativeTD = (
    evidences.filter(F.col("datasourceId") == "chembl")
    .select("targetId", "diseaseId", "studyStopReason", "studyStopReasonCategories")
    .filter(F.array_contains(F.col("studyStopReasonCategories"), "Negative"))
    .groupBy("targetId", "diseaseId")
    .count()
    .withColumn("stopReason", F.lit("Negative"))
    .drop("count")
)

print("built negativeTD dataset")

bench2 = benchmark.join(
    rightTissue, on=["name", "bioSampleName"], how="left"
).withColumn(
    "rightTissue",
    F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
)

print("built bench2 dataset")

###### cut from here
print("looping for variables_study")
# List of columns to analyze
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

# Dictionary to store results
pivoted_dfs = {}

# Loop over the columns
for col in variables_study:
    window_spec = Window.partitionBy("targetId", "diseaseId", col).orderBy(
        F.col("pValueExponent").asc()
    )
    print(f"Processing: {col}")

    pivoted_df = (
        bench2.withColumn(
            "rightTissue",
            F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        )
        .withColumn(
            "isRightTissueSignalAgreed",
            F.collect_set(
                F.when(F.col("rightTissue") == "yes", F.col("agree_lowestPval"))
            ).over(window_spec),
        )
        .withColumn(
            "isSignalFromRightTissue",
            F.first(
                F.when(
                    F.col("AgreeDrug") == F.col("agree_lowestPval"),
                    F.col("rightTissue"),
                ),
                ignorenulls=True,
            ).over(window_spec),
        )
        .groupBy(
            "targetId",
            "diseaseId",
            "maxClinPhase",
            "rightTissue",
            "isRightTissueSignalAgreed",
            "isSignalFromRightTissue",
        )
        .pivot(col)  # Pivot the column dynamically
        .agg(F.collect_set("agree_lowestPval"))
        .join(negativeTD, on=["targetId", "diseaseId"], how="left")
        .withColumn(
            "PhaseT",
            F.when(F.col("stopReason") == "Negative", F.lit("yes")).otherwise(
                F.lit("no")
            ),
        )
        .withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        ))  # Collect unique values

    # Store the DataFrame in the dictionary
    pivoted_dfs[col] = pivoted_df

spark session created at 2025-05-27 06:51:58.577600
Analysis started on 2025-05-27 at  2025-05-27 06:51:58.577600


25/05/27 06:52:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


loaded files
loaded newColoc


loaded gwasComplete
loaded resolvedColloc
run temporary direction of effect
built drugApproved dataset


built chemblAssoc dataset
built benchmark dataset


load comparisons_df_iterative function
created full_data and lists
loaded rightTissue dataset
built negativeTD dataset
built bench2 dataset
looping for variables_study
Processing: projectId


Processing: biosampleName


Processing: rightStudyType


Processing: colocDoE


In [4]:
pivoted_dfs['projectId'].show()

+---------------+-------------+------------+-----------+-------------------------+-----------------------+-----------+----------+---------+---------------------+--------+---------+---+-----+----------+----------+------+------------+-------+--------+-----+--------------+------+-----------+-----------------+----------+-----------+------------+------+----+---------+----------+------+----------+------+-------------+--------------+---------------------+--------+-------+-----------+-----------+----------+-------+----------------+----------+------+------+--------+--------+--------+
|       targetId|    diseaseId|maxClinPhase|rightTissue|isRightTissueSignalAgreed|isSignalFromRightTissue|Alasoo_2018|Aygun_2021|BLUEPRINT|Bossini-Castillo_2019|BrainSeq|Braineac2|CAP|CEDAR|CommonMind|Cytoimmgen|FUSION|Fairfax_2014|GENCORD|GEUVADIS| GTEx|Gilchrist_2021|HipSci|Jerber_2021|Kim-Hellmuth_2017|Lepik_2017|Nathan_2022|Nedelec_2016|OneK1K|PISA|Peng_2018|Perez_2022|PhLiPS|Quach_2016|ROSMAP|Randolph_2021|Sch

In [ ]:
disdic.items()

In [8]:
disdic.items()

dict_items([('Nedelec_2016', 'projectId'), ('BrainSeq', 'projectId'), ('GEUVADIS', 'projectId'), ('GTEx', 'projectId'), ('Schmiedel_2018', 'projectId'), ('Bossini-Castillo_2019', 'projectId'), ('CEDAR', 'projectId'), ('ROSMAP', 'projectId'), ('FUSION', 'projectId'), ('Cytoimmgen', 'projectId'), ('Quach_2016', 'projectId'), ('HipSci', 'projectId'), ('OneK1K', 'projectId'), ('TwinsUK', 'projectId'), ('iPSCORE', 'projectId'), ('CommonMind', 'projectId'), ('Sun_2018', 'projectId'), ('Alasoo_2018', 'projectId'), ('Randolph_2021', 'projectId'), ('GENCORD', 'projectId'), ('Lepik_2017', 'projectId'), ('Braineac2', 'projectId'), ('UKB_PPP_EUR', 'projectId'), ('Kim-Hellmuth_2017', 'projectId'), ('BLUEPRINT', 'projectId'), ('Perez_2022', 'projectId'), ('van_de_Bunt_2015', 'projectId'), ('Young_2019', 'projectId'), ('Jerber_2021', 'projectId'), ('PISA', 'projectId'), ('CAP', 'projectId'), ('Aygun_2021', 'projectId'), ('Walker_2019', 'projectId'), ('Schwartzentruber_2018', 'projectId'), ('Peng_2018

In [3]:
pivoted_dfs['projectId'].select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.array_contains(F.col(x), "yes"), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}_only")
                    for x, value in [
                        (key, val) for key, val in disdic.items() if val == col
                    ]
                ]
            )
            + (
                [
                    F.when(
                        F.array_contains(F.col("isRightTissueSignalAgreed"), "yes"),
                        F.lit("yes"),
                    )
                    .otherwise(F.lit("no"))
                    .alias(f"{x}_isRightTissueSignalAgreed")
                    for x, value in [
                        (key, val) for key, val in disdic.items() if val == col
                    ]
                ]
            )
        ).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `GoF_protect` cannot be resolved. Did you mean one of the following? [`BrainSeq`, `FUSION`, `HipSci`, `Phase4`, `PhaseT`].;
'Project [targetId#1758, diseaseId#1911, maxClinPhase#6142, rightTissue#18104, isRightTissueSignalAgreed#18203, isSignalFromRightTissue#18255, Alasoo_2018#18575, Aygun_2021#18577, BLUEPRINT#18579, Bossini-Castillo_2019#18581, BrainSeq#18583, Braineac2#18585, CAP#18587, CEDAR#18589, CommonMind#18591, Cytoimmgen#18593, FUSION#18595, Fairfax_2014#18597, GENCORD#18599, GEUVADIS#18601, GTEx#18603, Gilchrist_2021#18605, HipSci#18607, Jerber_2021#18609, ... 31 more fields]
+- Project [targetId#1758, diseaseId#1911, maxClinPhase#6142, rightTissue#18104, isRightTissueSignalAgreed#18203, isSignalFromRightTissue#18255, Alasoo_2018#18575, Aygun_2021#18577, BLUEPRINT#18579, Bossini-Castillo_2019#18581, BrainSeq#18583, Braineac2#18585, CAP#18587, CEDAR#18589, CommonMind#18591, Cytoim

In [ ]:
bench2

In [19]:
####
### 
value='projectId'
window_spec = Window.partitionBy("targetId", "diseaseId",'projectId').orderBy(
        F.col("pValueExponent").asc()
    )

bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).groupBy('targetId','diseaseId','maxClinPhase').pivot('projectId').agg(F.collect_set('agree_lowestPval')).select(
            ["*"]
            + (
                [  ### single columns
                    F.when(F.array_contains(F.col(x), "yes"), F.lit("yes"))
                    .otherwise(F.lit("no"))
                    .alias(f"{x}_only")
                    for x,value  in [
                        (key, val) for key, val in disdic.items() if val == value
                    ]
                ]
            )).withColumn('Phase4', F.when(F.col('maxClinPhase')==4, F.lit('yes')).otherwise(F.lit('no'))).show()
                          
##).groupBy('GTEx_only').pivot('Phase4').count().show()

+---------------+-------------+------------+-----------+----------+---------+---------------------+--------+---------+---+-----+----------+----------+------+------------+-------+--------+-----+--------------+------+-----------+-----------------+----------+-----------+------------+------+----+---------+----------+------+----------+------+-------------+--------------+---------------------+--------+-------+-----------+-----------+----------+-------+----------------+-----------------+-------------+-------------+---------+-------------------+--------------------------+----------+-----------+-----------+---------------+---------------+-----------+-----------+------------+------------+---------------+-------------+----------------+------------------+------------+---------------+--------------+----------------+----------------------+--------------+---------------+---------------------+---------------+----------------+---------+--------+---------------+----------------+-------------------------

In [5]:
(bench2.withColumn(
            "rightTissue",
            F.when(F.col("rightTissue1") == "yes", F.lit("yes")).otherwise(F.lit("no")),
        )
        .withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        )
        .withColumn(
            "isRightTissueSignalAgreed",
            F.collect_set(
                F.when(F.col("rightTissue") == "yes", F.col("agree_lowestPval"))
            ).over(window_spec),
        )
        .withColumn(
            "isSignalFromRightTissue",
            F.first(
                F.when(
                    F.col("AgreeDrug") == F.col("agree_lowestPval"),
                    F.col("rightTissue"),
                ).otherwise(F.lit('no')),
                ignorenulls=True,
            ).over(window_spec),
        ).show())

+--------------------+--------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+----------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------------+--------------------+----------------------+----------+---------+--------------+---------+------------------+-------------------+-------------------------+-------------------+------------------+--------------------+-------------------+------------------+--------------------+---------------+--------------------+--------------+--------------------+-----------+------------+---------------+---------------+---------+------------+-----------+----------------+-------------------------+-----------------------+
|                name|       biosampleName|   biosampleId|       targetId|    dise

In [15]:
window_spec

In [ ]:
bench2.select('')

In [ ]:
bench2.groupBy('targetId','diseaseId').pivot('biosampleName').agg(F.collect_set(''))

+--------------------+--------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+----------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+---------------+---------------------+----------------+----------------------+----------+---------+--------------+---------+------------------+-------------------+-------------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+---------------+--------------------+--------------+--------------------+-----------+------------+---------------+---------------+---------+------------+-----------+
|                name|       biosampleName|   biosampleId|       targetId|    diseaseId|    leftStudyLocusId|        rightStudyId|   rightStudyLocusId|chromo

In [22]:
window_spec = Window.partitionBy("targetId", "diseaseId",'biosampleName').orderBy(
        F.col("pValueExponent").asc()
    )

bench2.withColumn('agree_tissue', F.when((F.col('rightTissue1')=='yes') & (F.col('AgreeDrug')=='yes'), F.lit('yes')).otherwise(F.lit('no'))).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "agree_lowestPval",
            F.first("agree_tissue", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).select('targetId','diseaseId','agree_lowestPval','Phase4').distinct().groupBy('agree_lowestPval').pivot('Phase4').count().show()

+----------------+---+---+
|agree_lowestPval| no|yes|
+----------------+---+---+
|              no| 96| 55|
|             yes| 48| 34|
+----------------+---+---+



In [23]:
bench2.show()

+--------------------+--------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+----------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+---------------+---------------------+----------------+----------------------+----------+---------+--------------+---------+------------------+-------------------+-------------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+---------------+--------------------+--------------+--------------------+-----------+------------+---------------+---------------+---------+------------+-----------+
|                name|       biosampleName|   biosampleId|       targetId|    diseaseId|    leftStudyLocusId|        rightStudyId|   rightStudyLocusId|chromo

In [25]:
bench2.select('targetId','diseaseId').distinct().count()

172

In [26]:
bench2.show()

+--------------------+--------------------+--------------+---------------+-------------+--------------------+--------------------+--------------------+----------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+---------------+---------------------+----------------+----------------------+----------+---------+--------------+---------+------------------+-------------------+-------------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+---------------+--------------------+--------------+--------------------+-----------+------------+---------------+---------------+---------+------------+-----------+
|                name|       biosampleName|   biosampleId|       targetId|    diseaseId|    leftStudyLocusId|        rightStudyId|   rightStudyLocusId|chromo

In [30]:
window_spec = Window.partitionBy("targetId", "diseaseId",'biosampleName','rightStudyType').orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('RightTissue_Agree', 'rightStudyType').pivot('Phase4').count().sort(F.col('rightStudyType').desc(),F.col('RightTissue_Agree').desc()).show(20)

+-----------------+--------------+---+----+
|RightTissue_Agree|rightStudyType| no| yes|
+-----------------+--------------+---+----+
|              yes|         tuqtl|201| 174|
|               no|         tuqtl|727| 860|
|              yes|          sqtl|  4|  12|
|               no|          sqtl|218| 237|
|              yes|        sceqtl|  9|  35|
|               no|        sceqtl| 37|  44|
|              yes|          pqtl| 56|  45|
|               no|          pqtl| 12|  66|
|              yes|          eqtl|157| 151|
|               no|          eqtl|887|1283|
+-----------------+--------------+---+----+



In [39]:
window_spec = Window.partitionBy("targetId", "diseaseId",'colocDoE').orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ), 
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('targetId', 'diseaseId','Phase4').pivot('colocDoE').agg(F.collect_set('agree_lowestPval')).show()

+---------------+-------------+------+-----------+-----------+
|       targetId|    diseaseId|Phase4|GoF_protect|LoF_protect|
+---------------+-------------+------+-----------+-----------+
|ENSG00000006071|  EFO_0000400|   yes|       [no]|      [yes]|
|ENSG00000006071|  EFO_0000537|    no|       [no]|      [yes]|
|ENSG00000006071|MONDO_0005148|   yes|       [no]|      [yes]|
|ENSG00000010310|  EFO_0000400|   yes|      [yes]|       [no]|
|ENSG00000010310|MONDO_0005148|   yes|      [yes]|       [no]|
|ENSG00000064989|  EFO_0001073|    no|         []|       [no]|
|ENSG00000065989|  EFO_0000540|   yes|         []|      [yes]|
|ENSG00000065989|  EFO_0000676|   yes|         []|      [yes]|
|ENSG00000065989|MONDO_0002406|    no|         []|      [yes]|
|ENSG00000066468|MONDO_0008315|    no|       [no]|         []|
|ENSG00000068078|  EFO_0000313|    no|       [no]|      [yes]|
|ENSG00000068078|MONDO_0001187|    no|       [no]|      [yes]|
|ENSG00000068078|MONDO_0004986|    no|       [no]|     

In [37]:
window_spec = Window.partitionBy("targetId", "diseaseId",'colocDoE').orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ), 
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('targetId', 'diseaseId','Phase4').pivot('colocDoE').agg(F.collect_set('agree_lowestPval')).groupBy('GoF_protect').pivot('Phase4').count().show()

+-----------+---+---+
|GoF_protect| no|yes|
+-----------+---+---+
|         []| 34| 21|
|       [no]| 62| 39|
|      [yes]| 10|  6|
+-----------+---+---+



In [38]:
window_spec = Window.partitionBy("targetId", "diseaseId",'colocDoE').orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ), 
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('targetId', 'diseaseId','Phase4').pivot('colocDoE').agg(F.collect_set('agree_lowestPval')).groupBy('LoF_protect').pivot('Phase4').count().show()

+-----------+---+---+
|LoF_protect| no|yes|
+-----------+---+---+
|         []| 24|  9|
|       [no]|  8|  8|
|      [yes]| 74| 49|
+-----------+---+---+



In [ ]:
window_spec = Window.partitionBy("targetId", "diseaseId").orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('agree_lowestPval', 'colocDoE').pivot('Phase4').count().sort(F.col('colocDoE').desc(),F.col('agree_lowestPval').desc()).show(20)

+----------------+-----------+----+----+
|agree_lowestPval|   colocDoE|  no| yes|
+----------------+-----------+----+----+
|             yes|LoF_protect|1348|1879|
|              no|LoF_protect|  76| 104|
|             yes|GoF_protect|  60|  84|
|              no|GoF_protect| 824| 840|
+----------------+-----------+----+----+



In [31]:
window_spec = Window.partitionBy("targetId", "diseaseId",'rightStudyType').orderBy(
        F.col("pValueExponent").asc()
    )
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('agree_lowestPval', 'rightStudyType').pivot('Phase4').count().sort(F.col('rightStudyType').desc(),F.col('agree_lowestPval').desc()).show(20)

+----------------+--------------+---+----+
|agree_lowestPval|rightStudyType| no| yes|
+----------------+--------------+---+----+
|             yes|         tuqtl|686| 767|
|              no|         tuqtl|242| 267|
|             yes|          sqtl| 39|  47|
|              no|          sqtl|183| 202|
|             yes|        sceqtl| 41|  79|
|              no|        sceqtl|  5|NULL|
|             yes|          pqtl| 59|  48|
|              no|          pqtl|  9|  63|
|             yes|          eqtl|475| 798|
|              no|          eqtl|569| 636|
+----------------+--------------+---+----+



In [16]:
window_spec = Window.partitionBy("targetId", "diseaseId",'biosampleName')
bench2.withColumn(
            "agree_lowestPval",
            F.first("AgreeDrug", ignorenulls=True).over(
                window_spec
            ),  ### ignore nulls aded 29.01.2025
            #### take directionality from lowest p value
        ).withColumn(
            "RightTissue_Agree",
            F.when((F.col('agree_lowestPval')=='yes') & (F.col('rightTissue1')=='yes'), F.lit('yes')).otherwise(F.lit('no')),
        ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy('RightTissue_Agree').pivot('Phase4').count().show()

+-----------------+----+----+
|RightTissue_Agree|  no| yes|
+-----------------+----+----+
|               no|1881|2486|
|              yes| 427| 421|
+-----------------+----+----+



In [2]:
#######
##
####
##
#######

pivoted_dfs['biosampleName'].show()

+---------------+-------------+------------+-----------+-------------------------+-----------------------+------------+------+----------------------------------+--------------------------------+-----------------------------------------------+-------------------------------+--------------------------------------+-------------------------------+-------------+------+----------------+---------------+--------------+-------------+--------+-------------------------+----------------------+---------------+-----+------------+----------------+-----------------+---------------+----------------------------------------------+----------+---------------+-------------------+------------------------------+-----------------------------------------------+---------------------------+-----------------------------+----------+-----------+--------------+--------------------------+----------+------------+-----+-----------------------------+-------------------+-------------------------+------------------------+-

In [10]:
benchmark.join(
    rightTissue, on=["name", "bioSampleName"], how="left"
).select('name','biosampleName','rightTissue1').distinct().toPandas().to_csv('gs://ot-team/jroldan/analysis/20250526_rightTissue.csv')

In [3]:
pivoted_dfs['projectId'].show(vertical=True)

-RECORD 0----------------------------------------------------------
 targetId                                        | ENSG00000006071 
 diseaseId                                       | EFO_0000400     
 maxClinPhase                                    | 4.0             
 rightTissue                                     | no              
 isRightTissueSignalAgreed                       | []              
 isSignalFromRightTissue                         | no              
 Alasoo_2018                                     | []              
 Aygun_2021                                      | []              
 BLUEPRINT                                       | []              
 Bossini-Castillo_2019                           | []              
 BrainSeq                                        | [no]            
 Braineac2                                       | []              
 CAP                                             | []              
 CEDAR                                          